## Imports and Initialisations

In [16]:
import nltk
nltk.download('stopwords')

from nltk.corpus import brown, stopwords
import xml.etree.ElementTree as ET

sents = brown.sents()
stopwords = set(stopwords.words('english'))
tree = ET.parse('../data/test/subtask1-homographic-test.xml')
puncts = []
test_sent = []

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arghyabhattacharya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
F = open("../data/test/subtask1-homographic-test.gold")
scores = []
for item in F.readlines():
    ans = item.split()[1]
    scores.append(ans)

print(scores)

['1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '0', '0', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '0',

In [ ]:
root = tree.getroot()
for text in root.getchildren():
    sent = list()
    for word in text.getchildren():
        sent.append(word.text)
    sent = str(sent)
    test_sent.append(sent)
    

In [ ]:
def make_ordered_pairs(sentence):
    ans = list()
    for index, word in enumerate(sentence):
        temp = index
        while(temp != len(sentence) - 1):
            ans.append((sentence[index], sentence[temp]))
            temp += 1
    return ans

In [ ]:
for sent in sents[:20]:
    filtered_sent = [w for w in sent if not w.lower() in stopwords]
    pairs = make_ordered_pairs(filtered_sent)
    print(pairs)